In [ ]:
# -*- coding:utf-8 -*-
# max866 2024/3/4
# chatGPT 產出code，
# 先把要比對的照片(test.jpg)讀入，程式從歷史影像平台API https://photo.ardswc.gov.tw/api/Media/ 逐一取出照片
# 將2張照片都縮小到100寬度比較，如果2張照片比例不同就回傳None
# 比對照片採用均方差（Mean Squared Error, MSE）
# 多執行緒執行


import cv2
import numpy as np
import requests
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor
import time


def resize_image(image, width=None, height=None):
    if width is None and height is None:
        return image
    h, w = image.shape[:2]
    if width is None:
        ratio = height / float(h)
        width = int(w * ratio)
    elif height is None:
        ratio = width / float(w)
        height = int(h * ratio)
    resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)
    return resized

def compare_images(img1, img2):
    resized_img1 = resize_image(img1, width=100)  # 設定target_image大小寬度為100
    resized_img2 = resize_image(img2, width=100)  # 設定reference_image大小寬度為100

    # 確保兩張圖片大小相同
    if resized_img1.shape != resized_img2.shape:
        # print("圖片大小不匹配")
        return None

    # 計算均方差
    mse = np.mean((resized_img1 - resized_img2) ** 2)
    similarity = 100 - mse  # 將 MSE 轉換為相似度，這裡為 100 減去 MSE

    return max(0, similarity)  # 確保相似度在 0 到 100 之間

def get_image_urls(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        file_names = [item['FileName'] for item in data]
        image_urls = [f"{file_name.replace('.xml', '.jpg')}" for file_name in file_names]
        return image_urls
    else:
        return []

def check_similarity(image_url, target_image):
    response = requests.get(image_url)
    reference_image = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)

    if reference_image is None:
        print("無法下載或讀取參考圖片：", image_url)
        return

    similarity = compare_images(target_image, reference_image)

    if similarity is not None and similarity > 30:  # 加入對相似度是否為 None 的檢查
        print("比對成功，相似度為: {}%".format(similarity))
        print("分享連結:", image_url)
    else:
        # print(f"{image_url}"+" 比對失敗，相似度為: {}%".format(similarity))
        pass

def main():
    api_url = "https://photo.ardswc.gov.tw/api/Media/"
    api_images = get_image_urls(api_url)

    if not api_images:
        print("無法找到API中的照片")
        return

    target_image_path = "test.jpg"
    target_image = cv2.imread(target_image_path)

    if target_image is None:
        print("無法讀取目標圖片")
        return

    with ThreadPoolExecutor(max_workers=5) as executor:  # 使用最多5個工作執行緒
        for image_url in api_images:
            executor.submit(check_similarity, image_url, target_image)

if __name__ == "__main__":
    startTime = time.time()
    main()
    finishTime = time.time()
    print(f'花費時間 : {finishTime - startTime}')
